In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
simgsingle = str(imgsingle)
smcts = str(mcts)
sbigtiff = str(bigtiff)

## nima_io.read

| function   |  time | Pos in md | extra md | no errors | note                     |
|------------|-------|-----------|----------|-----------|--------------------------|
| read       |  4.68 | yes       | no       | no        |                          |
| read2      |  12.4 | no        | yes      | no        |                          |
| read_wrap  |     - | yes       | no       | yes       | It calls read            |
|----
| read_pims  |  2.57 | yes       | no       | yes       | extra pims DIMS          |
| read_jpype | 0.263 | yes       | yes      | yes       |                          |

Summary:

- The Jpype approach stands out for its thoroughness and performance.
- Scyjava, built on Jpype, is now superseding it.
- Functions like read_inf and read_bf may be candidates for removal from the convenient library.

In the following, various bioformats implementations are explored. While python-bioformats produces a verbose warning log through python-javabridge, the numerous metadata linked to individual planes are not returned by the external library. Hence, I decided to develop a small library to handle additional (neglected) metadata, such as acquisition stage position (essential for reconstructing tiled images) and illumination and emission settings.

### read_jpype

In [ ]:
%timeit ir.read_jpype(simg_void_tile)
%memit ir.read_jpype(simg_void_tile)

rjpype = ir.read_jpype(simg_void_tile)

In [ ]:
(core_md_jpype, (wr_jpype, type, extra_md_jpype)) = rjpype

In [ ]:
read_dictionary = np.load("extra_md_read2.npy", allow_pickle=True).item()

sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
sorted_dictJ = {k: extra_md_jpype[k] for k in sorted(extra_md_jpype.keys())}

In [ ]:
sorted(extra_md_jpype.keys()) == sorted(read_dictionary.keys())

In [ ]:
read_dictionary.keys() - extra_md_jpype.keys()

In [ ]:
read_dictionary["Format"], read_dictionary["Creator"]

In [ ]:
read_dictionary.pop("Format")
read_dictionary.pop("Creator")

In [ ]:
sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
for k in sorted_dict:
    if not sorted_dict[k] == sorted_dictJ[k]:
        print(k)

In [ ]:
print(
    read_dictionary["ChannelLightSourceSettingsAttenuation"],
    extra_md_jpype["ChannelLightSourceSettingsAttenuation"],
)
print(read_dictionary["PixelsBigEndian"], extra_md_jpype["PixelsBigEndian"])
print(read_dictionary["PixelsInterleaved"], extra_md_jpype["PixelsInterleaved"])

In [ ]:
f = ir.stitch(core_md_jpype, wr_jpype, c=1, t=0)
skimage.io.imshow(f)

In [ ]:
core_md_jpype["series"][0]["PositionXYZ"]

### PIMS

Which is currently unable to download loci_tools.jar.

**I really like the frame metadata t_s, x_um, y_um and z_um.
Every array (2D, 3D, ..., n-D) having those metadata in common are contained in the Frame obj: a numpy array with metadata(dict) and frame_no(int).**

Are fs.bundle_axes (fs.frame_shape), fs.iter_axes and fs.default_coords overcomplicated?

Anyway: iter=0 == iter=n which is at least unexpected.

In [ ]:
%timeit ir.read_pims(img_void_tile)
%memit ir.read_pims(img_void_tile)

rpims = ir.read_pims(img_void_tile)
rpims

### Update bioformats to scyjava

So to remove the annoying long warning, find the jar for loci and start stop JVM properly.

In [ ]:
bioformats.ImageReader(str(img_tile))

In [ ]:
# bioformats_jar
import jpype
import scyjava

scyjava.config.endpoints.append("ome:formats-gpl:6.7.0")
scyjava.start_jvm()
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")


def hello():
    JOptionPane = scyjava.jimport("javax.swing.JOptionPane")
    JOptionPane.showMessageDialog(None, "Hello world")


# jpype.setupGuiEnvironment(hello)

In [ ]:
# rdr = loci.formats.ChannelSeparator(loci.formats.ChannelFiller())
rdr = loci.formats.ImageReader()

In [ ]:
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(simg_void_tile)
xml_md = rdr.getMetadataStore()

In [ ]:
# sr = image_reader.getSeriesCount()
md, mdd = ir.get_md_dict(xml_md, simg_void_tile)

In [ ]:
# md['Format'] = rdr.format
# new core_md
# core_md = init_metadata(md["ImageCount"][0][1], rdr.format)
core_md = ir.init_metadata(md["ImageCount"][0][1], rdr.getFormat())

In [ ]:
core_md

In [ ]:
scyjava.jvm_started()

### TileSticher
   #+begin_src jupyter-python

     filepath = img_tile
     rdr = bioformats.formatreader.make_image_reader_class()()
     rdr.allowOpenToCheckType(True)

     clsOMEXMLService = javabridge.JClassWrapper('loci.formats.services.OMEXMLService')
     serviceFactory = javabridge.JClassWrapper('loci.common.services.ServiceFactory')()
     service = serviceFactory.getInstance(clsOMEXMLService.klass)
     metadata = service.createOMEXMLMetadata()
     rdr.setMetadataStore(metadata)
     rdr.setId(filepath)



     ts = javabridge.JClassWrapper('loci.formats.TileStitcher')(rdr)
     cs = javabridge.JClassWrapper('loci.formats.ChannelSeparator')(rdr)
     ##is_tile = ts.makeTileStitcher(rdr)
   #+end_src

   #+RESULTS:


   #+begin_src jupyter-python

     rdr.setId(filepath)

     s = 0
     root = metadata.getRoot()
     first_image = root.getImage(s)
     pixels = first_image.getPixels()

     # The plane data isn't in the planes, it's in the tiff data
     for idx in range(pixels.sizeOfTiffDataList()):
         tiffData = pixels.getTiffData(idx)
         c = tiffData.getFirstC().getValue().intValue()
         t = tiffData.getFirstT().getValue().intValue()
         print("TiffData: c=%d, t=%d" % (c, t))
   #+end_src

#+RESULTS:
#+begin_example
  TiffData: c=0, t=0
  TiffData: c=1, t=0
  TiffData: c=2, t=0
  TiffData: c=3, t=0
  TiffData: c=0, t=1
  TiffData: c=1, t=1
  TiffData: c=2, t=1
  TiffData: c=3, t=1
  TiffData: c=0, t=2
  TiffData: c=1, t=2
  TiffData: c=2, t=2
  TiffData: c=3, t=2
#+end_example



## ImageIO

Started comparing the available options to read microscopy file.

- mosaic

In [ ]:
from imageio.v3 import imread

%timeit imread(img_void_tile, index=13)
%memit imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

In [ ]:
i.nbytes, 512**2 * 3 * 4 * 2

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [ ]:
from aicsimageio import AICSImage

i = AICSImage(img_void_tile)
# i = AICSImage(img_void_tile, reconstruct_mosaic=True)
# i_lif = AICSImage(lif)

In [ ]:
i.ome_metadata

In [ ]:
i.scenes

In [ ]:
i.get_dask_stack()

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [ ]:
from dask_image.imread import imread

i = imread(img_void_tile)

In [ ]:
i

In [ ]:
imread(lif)

Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to install.

**pyimagej** need conda?

## bioio-bioformats

In [ ]:
import bioio_ome_tiled_tiff

In [ ]:
bioio_ome_tiled_tiff.Reader(str(img_tile))

In [ ]:
import bioio_bioformats

im = bioio_bioformats.Reader(img_void_tile)
bioio_bioformats.ReaderMetadata(img_void_tile)

In [ ]:
im.xarray_dask_data

In [ ]:
i = bioio_bioformats.Reader(img_tile)
i.data.shape

In [ ]:
i.xarray_dask_data.attrs["processed"]

In [ ]:
import nima_io.read as ir

In [ ]:
ir.bioformats.get_omexml_metadata(img_tile)

In [ ]:
unp = i.xarray_dask_data.attrs["unprocessed"]
?ir.get_md_dict

In [ ]:
i.ome_metadata

In [ ]:
stk = i.get_dask_stack()

In [ ]:
stk.A

## bfio

In [ ]:
import bfio

bfio.BioReader(img_void_tile)

In [ ]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

In [ ]:
rdr.metadata

In [ ]:
rdr.ps_x

In [ ]:
rdr.close()

## PIMS

In [ ]:
import pims

%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

## PyOMETiff

In [ ]:
import pyometiff

%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

In [ ]:
%timeit r = rdr.read()
res = rdr.read()

In [ ]:
res[2]

In [ ]:
pyometiff.OMETIFFReader._get_metadata_template()

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping

- impy
- napari.read
- pycromanager
- microscope
- python-microscopy